# IMDb（Internet Movie Database）DataLoader作成

在这个文件中，你可以使用IMDb(互联网电影数据库)的数据来创建数据分析(0:消极，1:积极)的二值类数据。


※　所有文件都是以在Ubuntu中运行为前提的。请注意在Windows等字符代码不同的环境下运行。

#  学习目标

1.	掌握根据文本格式的文件数据创建 tsv 文件以及使用 torchtext 编写 DataLoader 的方法。

# 事前准备
根据书籍的指示，准备本章要使用的数据。


# 1. 将IMDb 数据集转换为tsv 格式

下载Dataset。

※ 虽然在torchtext中有可以使用IMDb的标准函数，但是这次从0开始制作，即使在没有准备数据集的情况下也可以使用。

http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

5万个数据(train,test都是2.5万个)。文件名由数据id和rating(1-10)决定。

rate是10比较好。4以下是negative, 7以上是positive。



In [1]:
# 保存为tsv格式的文件
import glob
import os
import io
import string

# 创建训练数据的tsv文件
f = open('./data/IMDb_train.tsv', 'w', encoding="utf-8")

path = './data/aclImdb/train/pos/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        #如果包含制表符就删除
        text = text.replace('\t', " ")

        text = text+'\t'+'1'+'\t'+'\n'
        f.write(text)

path = './data/aclImdb/train/neg/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # 如果包含制表符就删除
        text = text.replace('\t', " ")

        text = text+'\t'+'0'+'\t'+'\n'
        f.write(text)

f.close()

In [2]:
# 创建测试数据
f = open('./data/IMDb_test.tsv', 'w', encoding="utf-8")

path = './data/aclImdb/test/pos/'

for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # 如果包含制表符就删除
        text = text.replace('\t', " ")

        text = text+'\t'+'1'+'\t'+'\n'
        f.write(text)


path = './data/aclImdb/test/neg/'

for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # 如果包含制表符就删除
        text = text.replace('\t', " ")

        text = text+'\t'+'0'+'\t'+'\n'
        f.write(text)

f.close()

# 2. 定义预处理和分词处理函数

In [3]:
import string
import re

# 将下列符号替换成空格符（除了句号和逗号之外）
# punctuation就是标点符号的意思
print("分隔符：", string.punctuation)
# !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~

# 预处理
def preprocessing_text(text):
    text = re.sub('<br />', '', text)

    # 将逗号和句号以外的标点符号替换成空格符
    for p in string.punctuation:
        if (p == ".") or (p == ","):
            continue
        else:
            text = text.replace(p, " ")

    # 在句号和逗号前后插入空格符
    text = text.replace(".", " . ")
    text = text.replace(",", " , ")
    return text


# 用空格将单词隔开（这里的数据是英文的，用空格进行分隔）
def tokenizer_punctuation(text):
    return text.strip().split()


# 定义用于预处理和分词处理的函数
def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)
    ret = tokenizer_punctuation(text)
    return ret


# 确认执行结果
print(tokenizer_with_preprocessing('I like cat.'))

分隔符： !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['I', 'like', 'cat', '.']


# 创建DataLoader

In [4]:
# 定义在读取数据时，对读取内容所做的处理
import torchtext

# 同时准备文章和标签两种字段
max_length = 256
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing, use_vocab=True,
                            lower=True, include_lengths=True, batch_first=True, fix_length=max_length, init_token="<cls>", eos_token="<eos>")
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

# 各个参数的含义如下
# init_token：在所有的文章中，插入在开头位置的单词
# eos_token： 在所有的文章中，插入在结尾位置的单词

In [5]:
#从data文件夹中读取各个tsv文件
train_val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='./data/', train='IMDb_train.tsv',
    test='IMDb_test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])

#确认执行结果
print('训练和验证数据的数量', len(train_val_ds))
print('第一个训练和验证的数据', vars(train_val_ds[0]))

训练和验证数据的数量 25000
第一个训练和验证的数据 {'Text': ['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy', '.', 'it', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life', ',', 'such', 'as', 'teachers', '.', 'my', '35', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'bromwell', 'high', 's', 'satire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', 'teachers', '.', 'the', 'scramble', 'to', 'survive', 'financially', ',', 'the', 'insightful', 'students', 'who', 'can', 'see', 'right', 'through', 'their', 'pathetic', 'teachers', 'pomp', ',', 'the', 'pettiness', 'of', 'the', 'whole', 'situation', ',', 'all', 'remind', 'me', 'of', 'the', 'schools', 'i', 'knew', 'and', 'their', 'students', '.', 'when', 'i', 'saw', 'the', 'episode', 'in', 'which', 'a', 'student', 'repeatedly', 'tried', 'to', 'burn', 'down', 'the', 'school', ',', 'i', 'immediately', 'recalled', '.', '.', '.', '.', '.', '.', '.', '.', '.', 'at', '.', '.', 

In [6]:
import random
#使用torchtext.data.Dataset的split函数将Dataset切分为训练数据和验证数据

train_ds, val_ds = train_val_ds.split(
    split_ratio=0.8, random_state=random.seed(1234))

#确认执行结果
print('训练数据的数量', len(train_ds))
print('验证数据的数量', len(val_ds))
print('第一个训练数据', vars(train_ds[0]))

训练数据的数量 20000
验证数据的数量 5000
第一个训练数据 {'Text': ['this', 'movie', 'has', 'got', 'to', 'be', 'one', 'of', 'the', 'worst', 'i', 'have', 'ever', 'seen', 'make', 'it', 'to', 'dvd', 'the', 'story', 'line', 'might', 'have', 'clicked', 'if', 'the', 'film', 'had', 'more', 'funding', 'and', 'writers', 'that', 'would', 'have', 'cut', 'the', 'nonsense', 'and', 'sickly', 'scenes', 'that', 'i', 'highly', 'caution', 'parents', 'on', '.', '.', '.', '.', 'but', 'the', 'story', 'line', 'is', 'like', 'a', 'loose', 'cannon', '.', 'if', 'there', 'was', 'such', 'a', 'thing', 'as', 'a', 'drive', 'thru', 'movie', 'maker', 'this', 'one', 'would', 'have', 'sprung', 'from', 'that', '.', 'it', 'reminded', 'me', 'a', 'lot', 'of', 'the', 'quickie', 'films', 'that', 'were', 'put', 'out', 'in', 'the', '1960', 's', ',', 'poor', 'script', 'writing', 'and', 'filming', '.', 'the', 'only', 'sensible', 'characters', 'in', 'the', 'whole', 'movie', 'was', 'the', 'bartender', 'and', 'beaver', '.', 'the', 'rest', 'of', 'the', 'fi

# 创建词汇表

In [7]:
#使用torchtext读取作为单词向量的已经完成学习的英语模型

from torchtext.vocab import Vectors

english_fasttext_vectors = Vectors(name='data/wiki-news-300d-1M.vec')


#确认单词向量中的内容
print("1个单词向量的维数：", english_fasttext_vectors.dim)
print("单词数量：", len(english_fasttext_vectors.itos))

1个单词向量的维数： 300
单词数量： 999994


In [8]:
# 单词数量
TEXT.build_vocab(train_ds, vectors=english_fasttext_vectors, min_freq=10)

#确认词汇表中的向量
print(TEXT.vocab.vectors.shape)  #用 300 阶向量表示的 17916 个单词
TEXT.vocab.vectors

#对词汇表中的单词顺序进行确认
TEXT.vocab.stoi

torch.Size([17915, 300])


defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x00000280C5E95040>>,
            {'<unk>': 0,
             '<pad>': 1,
             '<cls>': 2,
             '<eos>': 3,
             'the': 4,
             '.': 5,
             ',': 6,
             'and': 7,
             'a': 8,
             'of': 9,
             'to': 10,
             'is': 11,
             'it': 12,
             'in': 13,
             'i': 14,
             'this': 15,
             'that': 16,
             's': 17,
             'was': 18,
             'as': 19,
             'for': 20,
             'movie': 21,
             'with': 22,
             'but': 23,
             'film': 24,
             'you': 25,
             't': 26,
             'on': 27,
             'not': 28,
             'he': 29,
             'are': 30,
             'his': 31,
             'have': 32,
             'be': 33,
             'one': 34,
             'all': 35,
             'at': 36,
             'they':

In [9]:
train_dl = torchtext.data.Iterator(train_ds, batch_size=24, train=True)

val_dl = torchtext.data.Iterator(
    val_ds, batch_size=24, train=False, sort=False)

test_dl = torchtext.data.Iterator(
    test_ds, batch_size=24, train=False, sort=False)


#测试
batch = next(iter(val_dl))
print(batch.Text)
print(batch.Label)

(tensor([[   2,    8,  870,  ...,    1,    1,    1],
        [   2, 4487,    6,  ...,    4, 7480,    3],
        [   2,    4, 1859,  ...,    1,    1,    1],
        ...,
        [   2,  103, 2335,  ..., 1997,    6,    3],
        [   2,   42,  383,  ...,    1,    1,    1],
        [   2,    4,  895,  ...,  803,    6,    3]]), tensor([120, 256, 248, 256, 195, 256, 256, 144, 256, 256, 194, 256, 256, 180,
        226, 256, 256, 256,  73, 137, 160, 256, 130, 256]))
tensor([1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0])


这里采取了在深度学习的模型中根据单词 ID 查找对应的向量表现的实现方式。

实现代码保存在utils文件夹里的dataloader.py文件中。